In [1]:
import re
import warnings

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_validate, KFold
from statsmodels.iolib.summary2 import summary_col
from statsmodels.regression.mixed_linear_model import MixedLM
from statsmodels.tools.eval_measures import aicc
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from sklearn.preprocessing import OneHotEncoder

warnings.simplefilter("ignore", ConvergenceWarning)

data = (
    pd.read_parquet("balanced_data.parquet")
    # .merge(pd.read_excel("land_cover.xlsx"), on=["GID_0", "NAME", "YEAR"])
    # .rename(columns={"NTL_sum": "NTL"})
)
enc = OneHotEncoder(sparse_output=False)
encoded = enc.fit_transform(data.index.get_level_values("GID_0").values.reshape(len(data), -1))
data[enc.get_feature_names_out()] = encoded
data["ALL"] = 1
train_data = data.query("GID_0 != 'UGA' | NAME == 'Uganda'").copy().reset_index()
test_data = data.query("GID_0 == 'UGA' & NAME != 'Uganda'").copy().reset_index()
y_true = data.loc[("UGA", "Uganda"), "GDP"]


def calc_r_squared(result, df):
    var_resid = result.scale
    var_random_effect = result.cov_re.sum().iloc[0]
    var_fixed_effect = result.predict(df).var()

    total_var = var_fixed_effect + var_random_effect + var_resid
    marginal_r2 = var_fixed_effect / total_var
    conditional_r2 = (var_fixed_effect + var_random_effect) / total_var
    return marginal_r2, conditional_r2


def predict(model):
    return (
        test_data[["YEAR"]]
        .merge(
            np.exp(model.predict(test_data)).rename("GDP"),
            left_index=True,
            right_index=True,
        )
        .groupby("YEAR")["GDP"]
        .sum()
    )

In [2]:
models = [
    smf.mixedlm(
        "np.log(GDP) ~ np.log1p(NTL)",
        train_data,
        groups="YEAR",
        re_formula="np.log1p(NTL)",
    ).fit(reml=False),
    smf.mixedlm(
        "np.log(GDP) ~ np.log1p(NTL) + np.log(POP)",
        train_data,
        groups="YEAR",
        re_formula="np.log1p(NTL)",
    ).fit(reml=False),
    smf.mixedlm(
        "np.log(GDP) ~ np.log1p(NTL) + np.log(POP) + np.log(COUNTRY_AREA) + np.log(REGIONS)",
        train_data,
        groups="YEAR",
        re_formula="np.log1p(NTL)",
    ).fit(reml=False),
    smf.ols(
        "np.log(GDP) ~ np.log1p(NTL) + np.log(POP) + np.log(COUNTRY_AREA) + np.log(REGIONS)",
        train_data,
    ).fit(),
]

s = summary_col(
    models,
    float_format="%.2f",
    model_names=[f"({i + 1})" for i in range(len(models))],
    info_dict={
        "Observations": lambda x: x.nobs,
        "No. of years": lambda _: len(train_data["YEAR"].unique()),
        "No. of countries": lambda _: len(train_data["GID_0"].unique()),
        "R^2": lambda x: calc_r_squared(x, train_data)[1]
        if isinstance(x.model, MixedLM)
        else x.rsquared,
        "AIC": lambda x: x.aic,
        "AICc": lambda x: aicc(
            x.llf, x.nobs, x.df_modelwc if isinstance(x.model, MixedLM) else x.df_model
        ),
    },
    stars=True,
    regressor_order=[
        "Intercept",
        "NTL",
        "np.log1p(NTL)",
        "np.log(POP)",
        "np.log(COUNTRY_GDP)",
        "np.log(COUNTRY_AREA)",
        "np.log(REGIONS)",
        "YEAR Var",
    ],
    drop_omitted=True,
)
# s.add_title(
#     "Statistical relationship between log(GDP), log1p(NTL), and covariates (log(POP), log(COUNTRY\_AREA), log(REGIONS)) at a national level"
# )
# as_latex = re.sub(
#     r"np\.([\w()\\]+)",
#     r"\1   ",
#     s.as_latex()
#     .replace(r"\label{}", r"\label{tab:summaries}")
#     .replace(".0000", " " * 5)
#     .replace(r"R^2  ", r"$R^2$"),
# )
# as_latex = re.sub(
#     r"YEAR Var\s+(.+&)\s+\\",
#     rf"Year as random effects& {'Yes': <10}& {'Yes': <9}& {'Yes': <10}& {'No': <11}\\\\\n{'Year (Intercept)': <22}\1 {'-': <11}\\",
#     as_latex,
# )
# note = r"""\multicolumn{5}{p{0.9\linewidth}}{%
# \rule{0pt}{1em}%
# The data for the years from 2013 to 2020 are used. Columns 1, 2, and 3 show results for linear mixed models fitted using restricted maximum likelihood estimation, while column 4 is only fitted with fixed effects. Best-fit model is shown in column 3 based on its highest $R^2$ and lowest AIC and AICc consistently. log(GDP), log-transformed gross domestic product; log(POP), log-transformed population count; log(COUNTRY\_AREA), log-transformed country area; log(REGIONS), log-transformed number of first-level administrative regions of country.\newline
# ***P < 0.001.}\\
# """
# as_latex = "\n".join(
#     [
#         note + line if line.startswith(r"\end{tabular}") else line
#         for line in as_latex.splitlines()
#         if "nan" not in line and not line.startswith("R-squared")
#     ]
# )


# as_latex = (
#     as_latex.replace(
#         r"\hline",
#         r"""\toprule
#  & \multicolumn{4}{c}{Dependent variable: log(GDP)}\\
# \cmidrule(l{3pt}r{3pt}){2-5}""",
#         1,
#     )
#     .replace(r"\hline", r"\midrule", 1)
#     .replace(r"\hline", r"\bottomrule", 1)
# )
# as_latex = re.sub(r" +& \(1\)", f"{'Independent variables': <22}& (1)", as_latex)

# with open("table.txt", "w") as f:
#     f.write(as_latex)
s

,(1),(2),(3),(4)
Intercept,17.62***,11.39***,-38.08***,-37.12***
,(0.17),(0.32),(2.63),(2.67)
np.log1p(NTL),0.58***,0.24***,0.39***,0.37***
,(0.02),(0.02),(0.02),(0.01)
np.log(POP),,0.62***,0.58***,0.60***
,,(0.03),(0.02),(0.02)
np.log(COUNTRY_AREA),,,1.63***,1.59***
,,,(0.09),(0.09)
np.log(REGIONS),,,1.31***,1.29***
,,,(0.04),(0.05)


In [3]:
mdf = models[2]

print(calc_r_squared(mdf, train_data))
y_pred = predict(mdf)
mape = mean_absolute_percentage_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
print(mape * 100, np.sqrt(mse) / y_true.mean() * 100)


def validate_plot(model, d):
    valid = np.exp(model.predict(d))
    print(mean_absolute_percentage_error(d["GDP"], valid) * 100)
    error = ((d["GDP"] - valid) / d["GDP"]).rename("ERR") * 100
    return d[["GID_0", "NAME", "YEAR"]].merge(error, left_index=True, right_index=True)


d = validate_plot(mdf, train_data)


(0.894308919134701, 0.922287924453604)
15.861500325280039 15.83422121302217
25.154524096829693


In [6]:
train_data.columns

Index(['GID_0', 'NAME', 'YEAR', 'NTL', 'logNTL', 'NTLc', 'POP', 'GDP', 'AREA',
       'LVL', 'COUNTRY_NTL', 'COUNTRY_logNTL', 'COUNTRY_NTLc', 'COUNTRY_POP',
       'COUNTRY_GDP', 'COUNTRY_AREA', 'REGIONS', 'x0_KEN', 'x0_MOZ', 'x0_TZA',
       'x0_UGA', 'ALL'],
      dtype='object')

In [20]:
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from lightgbm import LGBMRegressor
from sklearn.gaussian_process.kernels import RBF

kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))

cols = [
    "NTL",
    "logNTL",
    "NTLc",
    "POP",
    "LVL",
    "AREA",
    *enc.get_feature_names_out(),
    #"COUNTRY_POP",
    #"COUNTRY_AREA",
    #"REGIONS",
]

seed = 2023

kf = KFold(n_splits=5, shuffle=True, random_state=seed)

ml_models = {
    "RF": RandomForestRegressor(random_state=seed),
    "ADAB": AdaBoostRegressor(random_state=seed),
    "GB": GradientBoostingRegressor(random_state=seed),
    "XGB": XGBRegressor(random_state=seed),
    "XGBRF": XGBRFRegressor(random_state=seed),
    #"SVR": SVR(),
    "LR": LinearRegression(),
    #"ISO": IsotonicRegression(),
    # "LGBM": LGBMRegressor(random_state=seed),
    # "GP": GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9),
    "DT": DecisionTreeRegressor(random_state=seed),
}


# for i, (train_index, test_index) in enumerate(kf.split(train_data)):
#     t = train_data.iloc[train_index]
#     v = train_data.iloc[test_index]
#     regr = RandomForestRegressor(random_state=seed)
#     regr.fit(t[cols], t["GDP"])
#     print(f"Fold {i}:")
#     print(f"Num of train data: {len(t)}, Num of validate data: {len(v)}")
#     for i, g in v.groupby("GID_0"):
#         print(f"{i}: {mean_absolute_percentage_error(g['GDP'], regr.predict(g[cols])):%}")
#     for i, g in v.groupby("LVL"):
#         print(f"ADMIN{i}: {mean_absolute_percentage_error(g['GDP'], regr.predict(g[cols])):%}")
#     for i, g in v.groupby(["GID_0", "LVL"]):
#         print(f"{i}: {mean_absolute_percentage_error(g['GDP'], regr.predict(g[cols])):%}")
#     print(f"All train data: {mean_absolute_percentage_error(v['GDP'], regr.predict(v[cols])):%}")

for name, regr in ml_models.items():
    cv_result = cross_validate(regr, train_data[cols], train_data["GDP"], cv=kf, scoring=["r2", "neg_root_mean_squared_error"])
    print(f"{name: <10}", end="")
    print(cv_result["test_r2"], end="")
    print(
        -cv_result["test_neg_root_mean_squared_error"] / train_data["GDP"].mean() * 100
    )


RF        [0.99355807 0.99016701 0.99265273 0.98724979 0.99013862][15.83299496 29.47241003 28.15599768 37.72220262 22.09804006]
ADAB      [0.97679586 0.97764017 0.9799608  0.96744414 0.9784566 ][30.04952866 44.44337306 46.49949703 60.27717775 32.66193663]
GB        [0.99639443 0.99523301 0.99523103 0.98468308 0.98576431][11.84519091 20.52083184 22.68403357 41.3450998  26.5505824 ]
XGB       [0.99570825 0.99634579 0.99690715 0.97379138 0.98180392][12.9232631  17.96675025 18.26786401 54.08295332 30.01744811]
XGBRF     [0.99130167 0.98550018 0.98052607 0.97418047 0.98211021][18.39809759 35.78937917 45.83897155 53.68000096 29.76373251]
LR        [0.96731039 0.97056082 0.97715217 0.95226277 0.95148339][35.66643861 50.99594926 49.65129037 72.99060486 49.01511463]
DT        [0.99466169 0.98335545 0.99594363 0.93998488 0.99453564][14.4130801  38.3450055  20.92073709 81.84057775 16.44955885]
